### Feature Exploration
In this notebook we'll explore some of the features of the SDK, such as caching and pagination.

In [1]:
import os

from shroomdk import ShroomDK

YOUR_API_KEY = os.environ.get("SHROOMDK_API_KEY")

sdk = ShroomDK(YOUR_API_KEY)

### The Query
Get all the projects that Moonbird NFT minters have bought/sold over the past 30 days aggregated by total sales. 

```sql
with moonbirds_minters as (
	SELECT
    	distinct nft_to_address as addr
	FROM ethereum.core.ez_nft_mints
	WHERE 
  		nft_address = lower('0x23581767a106ae21c074b2276D25e5C3e136a68b')
	ORDER BY 1 DESC
),
sellers as (
  SELECT
  	s.nft_address as addr,
  	s.project_name,
  	count(distinct s.seller_address) as uniq_sellers,
  	count(1) as num_sales,
  	sum(price_usd) as usd_volume
  FROM ethereum.core.ez_nft_sales s
  INNER JOIN moonbirds_minters mm ON mm.addr = s.seller_address
  WHERE
      block_timestamp >= GETDATE() - interval'30 days'
  GROUP BY 1,2
  ORDER BY 1 DESC
),
buyers as (
  SELECT
  	s.nft_address as addr,
  	s.project_name,
  	count(distinct s.buyer_address) as uniq_buyers,
  	count(1) as num_sales,
  	sum(price_usd) as usd_volume
  FROM ethereum.core.ez_nft_sales s
  INNER JOIN moonbirds_minters mm ON mm.addr = s.buyer_address
  WHERE
      block_timestamp >= GETDATE() - interval'30 days'
  GROUP BY 1,2
  ORDER BY 1 DESC
),
addresses as (
  SELECT
  	coalesce(sellers.addr, buyers.addr) as addr,
  	coalesce(sellers.project_name, buyers.project_name) as project_name
  FROM sellers
  LEFT OUTER JOIN buyers ON buyers.addr = sellers.addr
)
SELECT
	addresses.addr,
	addresses.project_name,
	coalesce(addresses.project_name, addresses.addr) as project,
	coalesce(s.uniq_sellers, 0) as uniq_sellers,
	coalesce(b.uniq_buyers, 0) as uniq_buyers,
	coalesce(s.num_sales, 0) as sale_count,
	coalesce(b.num_sales, 0) as buy_count,
	coalesce(s.num_sales, 0) + coalesce(b.num_sales, 0) as total_sale_count,
	coalesce(s.usd_volume, 0) as sale_vol_usd,
	coalesce(b.usd_volume, 0) as buy_vol_usd,
	coalesce(s.usd_volume, 0) + coalesce(b.usd_volume, 0) as total_vol_usd
FROM addresses 
LEFT OUTER JOIN buyers b ON b.addr = addresses.addr
LEFT OUTER JOIN sellers s ON s.addr = addresses.addr
order by 9 desc
```

In [2]:
# Read the above sql from a file and assign to `sql` variable
sql_statement = open("sql/minter_sales_agg.sql").read()

Execute the query

In [3]:
query_result_set = sdk.query(sql_statement)

records = query_result_set.records

if not records:
    raise Exception("No records found")

Caching

In [7]:
# Caching Example

import time

start = time.time()

sdk.query(sql_statement + " --- this is a different comment ", ttl_minutes=1)

print(f"query ran in {time.time() - start} seconds")

query ran in 0.31488895416259766 seconds


In [8]:
records[0].keys()

dict_keys(['addr', 'project_name', 'project', 'uniq_sellers', 'uniq_buyers', 'sale_count', 'buy_count', 'total_sale_count', 'sale_vol_usd', 'buy_vol_usd', 'total_vol_usd'])

In [11]:
query_result_set.run_stats.record_count

3141

Plot the results

In [9]:
import pandas as pd
import plotly.express as px

# Easily create a data frame from the query result set
df = pd.DataFrame(records)

fig = px.scatter(
    df, 
    x="uniq_buyers", 
    y="uniq_sellers",
	size="total_vol_usd", 
    color="total_sale_count",
    hover_name="project", 
    size_max=60
)

fig.show()

#### Pagination

In [14]:
# Pagination

query_result_set = sdk.query(sql_statement, page_number=20, page_size=10)
records = query_result_set.records
if not records:
    raise Exception("no records returned from query")

df = pd.DataFrame(records)

display(df)

fig = px.scatter(
    df, 
    x="uniq_buyers", 
    y="uniq_sellers",
	size="total_vol_usd", 
    color="total_sale_count",
    hover_name="project", 
    size_max=30
)

fig.show()


,addr,project_name,project,uniq_sellers,uniq_buyers,sale_count,buy_count,total_sale_count,sale_vol_usd,buy_vol_usd,total_vol_usd
0,0xd21818b6052df69eed04e9b2af564b75140aacb7,None,0xd21818b6052df69eed04e9b2af564b75140aacb7,3,0,3,0,3,14987.21,0.00,14987.21
1,0xfe8c6d19365453d26af321d0e8c910428c23873f,creepz,creepz,6,4,9,4,13,14953.23,6356.54,21309.77
2,0x5af0d9827e0c53e4799bb226655a1de152a425a5,milady,milady,10,3,12,4,16,14646.87,4934.55,19581.42
3,0x59468516a8259058bad1ca5f8f4bff190d30e066,invisible friends,invisible friends,3,3,3,3,6,14640.37,15633.10,30273.47
4,0xd7b397edad16ca8111ca4a3b832d0a5e3ae2438c,None,0xd7b397edad16ca8111ca4a3b832d0a5e3ae2438c,6,2,7,3,10,14624.99,5363.49,19988.48
5,0x8dc7b6ec6fafa36085ee9ec8e39112428d3360aa,None,0x8dc7b6ec6fafa36085ee9ec8e39112428d3360aa,14,3,17,7,24,14399.74,2193.51,16593.25
6,0xc92ceddfb8dd984a89fb494c376f9a48b999aafc,creature world,creature world,16,7,20,7,27,13775.12,4105.75,17880.87
7,0x6fc3ad6177b07227647ad6b4ae03cc476541a2a0,DarkEchelon,DarkEchelon,3,0,3,0,3,13533.70,0.00,13533.70
8,0x11450058d796b02eb53e65374be59cff65d3fe7f,the shiboshis,the shiboshis,3,1,10,3,13,13509.07,2593.72,16102.79
9,0xdedf88899d7c9025f19c6c9f188deb98d49cd760,None,0xdedf88899d7c9025f19c6c9f188deb98d49cd760,11,8,17,9,26,13335.80,6987.53,20323.33
